In [2]:
using PyPlot
using MLDatasets, LinearAlgebra, Statistics, Images

train_x, train_y = MNIST.traindata()
test_x, test_y = MNIST.testdata()
;

In [ ]:
relu(x) = max(0, x)
sigmoid(x) = 1 / (1 + exp(-x))
relu_derivative(y) = ifelse(y > 0, 1, 0)
sigmoid_derivative(y) = y * (1 - y)

function random_initialization(layer_dims)
    parameters = []
    L = length(layer_dims) - 1
    for i in 1:L
        W = randn(layer_dims[i + 1], layer_dims[i]) * sqrt(2 / layer_dims[i])
        b = zeros(layer_dims[i + 1], 1)
        push!(parameters, [W, b])
    end
    parameters
end

function moment_initialization(parameters)
    V = []
    for W, b in parameters
        

function forward(X, parameters)
    caches = [X]
    A = X
    L = length(parameters)
    for i in 1:L
        W, b = parameters[i]
        Z = W * A .+ b
        A = ifelse(i == L, sigmoid.(Z), relu.(Z))
        push!(caches, A)
    end
    caches
end

function backward(Y, parameters, caches)
    grads = []
    L = length